https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads

In [45]:
#!pip install convertbng

In [46]:
import os
import re
import time

# import gdal
import json
import datetime
from json import dumps
from copy import deepcopy
from convertbng.util import convert_lonlat

from multiprocessing import Pool, cpu_count

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.speedups
# import swifter

import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import clear_output

import warnings

warnings.filterwarnings("ignore")
pd.set_option("max_colwidth", None)

### Configurations

In [47]:
cfg = dict()

cfg["download data"] = False
cfg["to save"] = True
cfg["process geodata"] = False

cfg["start_year"] = 1995
cfg["end_year"] = 2024
cfg["Years"] = list(range(cfg["start_year"], cfg["end_year"] + 1))

cfg["geodata dir"] = "../input/geoData"
cfg["distribution dir"] = "../input/Distribution"
cfg["houseprice dir"] = "../input/HousePriceData"
cfg["school dir"] = "../input/SchoolData"

cfg["pp_raw dir"] = os.path.join(cfg["houseprice dir"], "Raw")
cfg["pp_processed dir"] = os.path.join(cfg["houseprice dir"], "Processed")

cfg["appData dir"] = "../appData"
cfg["assets dir"] = "../assets"

cfg["process_raw_pp"] = True
# cfg['process_raw_pp']   = False

cfg["raw price files"] = ["pp-2024.csv"]
# cfg['raw price files']  = ['pp-1995-2017.csv', 'pp-2018.csv', 'pp-2019.csv', 'pp-2020.csv', 'pp-2021.csv', 'pp-2022.csv']]

cfg["year"] = [2024]  # Additonal check - only process years in this list
# cfg['year']             = list(range(1995, 2021)) #Additonal check - only process years in this list

cfg["price_threshold"] = 10000  # Filter out transactions below this value

cfg["tolerance"] = (
	0.001  # Initial Tolerance threshold for shapely polygons simplification
)
cfg["max discrepancy"] = (
	5  # Maximum discrepancy (%) allowed for changes in polygon area after simplification.
)

cfg["regions_lookup"] = {
	"North East": "North England",
	"North West": "North England",
	"East Midlands": "Midlands",
	"West Midlands": "Midlands",
	"Greater London": "Greater London",
	"South East": "South East",
	"South West": "South West",
	"Wales": "Wales",
	"Scotland": "Scotland",
	"Northern Ireland": "Northern Ireland",
}

cfg["Regions"] = [
	"North England",
	"South East",
	"Greater London",
	"Midlands",
	"South West",
	"Wales",
	"Scotland",
]

#### Directory structure

In [48]:
"""
- appData (dir.)
- Data Pre-processing.ipynb
- input
    |-- Distribution 
    |-- geoData
    |-- SchoolData
    |-- HousePriceData
            |-- Raw
            |-- Processed
"""

os.makedirs(cfg["appData dir"], exist_ok=True)
os.makedirs(cfg["pp_processed dir"], exist_ok=True)

In [49]:
t0 = time.time()

#### Download Data

In [50]:
# Download and unzip postcode shape files
if cfg["download data"]:
	!wget https://www.opendoorlogistics.com/wp-content/uploads/Data/UK-postcode-boundaries-Jan-2015.zip
	!unzip UK-postcode-boundaries-Jan-2015.zip -d ../input
	!rm UK-postcode-boundaries-Jan-2015.zip
	clear_output()

In [51]:
# Download postcode data
if cfg["download data"]:
	!wget https://www.freemaptools.com/download/full-postcodes/ukpostcodes.zip
	!unzip ukpostcodes.zip -d ../input/geoData
	!rm ukpostcodes.zip
	clear_output()

##### Uncomment to download all files (The 1995 - 2017 file is 3.7GB)

In [52]:
# Download and House paid-price files
if cfg["download data"]:
	!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2020.csv -P ../input/HousePriceData/Raw/
	!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv -P ../input/HousePriceData/Raw/
	!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv -P ../input/HousePriceData/Raw/
	!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv -P ../input/HousePriceData/Raw/
	clear_output()

### Post Code Data - Lookup dictionaries to speed up processing

In [53]:
postcodes_df = pd.read_csv(os.path.join(cfg["geodata dir"], "ukpostcodes.csv"))

postcodes = dict()
for postcode, latitude, longitude in postcodes_df[
	["postcode", "latitude", "longitude"]
].values:
	postcodes[postcode] = [latitude, longitude]

# -----------------------------------------------------#

postcode_region_df = pd.read_csv(
	os.path.join(cfg["geodata dir"], "PostCode Region.csv")
)

postcode_region = dict()
for prefix, region in postcode_region_df[["Prefix", "Region"]].values:
	postcode_region[prefix] = cfg["regions_lookup"][region]

### House Price Data

#### Processing raw data

In [54]:
def lookup_postcode(postcodes, x):
	if x in postcodes:
		return postcodes[x]
	else:
		return ""


pattern = re.compile(r"\d")


def lookup_region(postcode_region, x, pattern=pattern):
	m = pattern.search(x)
	if m is None:
		return ""
	else:
		x = x[: m.start()]
		if x in postcode_region:
			return postcode_region[x]
		else:
			return ""

In [55]:
def clean_pp_df(df, postcodes, postcode_region):
	col = {
		1: "Price",
		2: "Date",
		3: "Post Code",
		4: "Property Type",
		5: "Old/New",
		6: "Duration",
	}

	df.rename(columns=col, inplace=True)
	df.fillna("", inplace=True)
	df["Address"] = (
		df[7]
		+ " "
		+ df[8]
		+ " "
		+ df[9]
		+ " "
		+ df[10]
		+ " "
		+ df[11]
		+ " "
		+ df[12]
		+ " "
		+ df[13]
	)
	df["Address"] = df["Address"].apply(lambda x: " ".join(x.split()))

	cols_to_drop = [col for col in df.columns if isinstance(col, int)]
	df.drop(cols_to_drop, axis=1, inplace=True)

	# Drop suspiciously low house price data: (Note: These have Property Type "Other". What is it?)
	df = df.loc[df.Price > cfg["price_threshold"]]

	# Exclude property type Other (O)
	df = df.loc[df["Property Type"] != "O"]

	# Sort by Date:
	df.sort_values(by=["Date"], inplace=True, ignore_index=True)

	# Get Latitude and Longitude by Post Code:
	df["Post Code Coords"] = df["Post Code"].apply(
		lambda x: lookup_postcode(postcodes, x)
	)

	# Get year-month:
	df["Year-Month"] = df["Date"].apply(lambda s: s[:7])
	df["Year"] = df["Date"].apply(lambda s: s[:4])
	df["Month"] = df["Date"].apply(lambda s: s[5:7])

	# Get Post code sector
	df["Sector"] = df["Post Code"].apply(lambda s: s[: s.find(" ") + 2])

	# Get Region
	df["Region"] = df["Post Code"].apply(lambda s: lookup_region(postcode_region, s))

	return df

In [56]:
%%time


def process_and_save_houseprice(infile, postcodes, postcode_region):
	print(f"Processing {infile}")

	df = pd.read_csv(os.path.join(cfg["pp_raw dir"], infile), header=None)
	df = clean_pp_df(df, postcodes, postcode_region)
	print(f"Number of transactions in {infile}: {len(df):,}\n")

	for year in df.Year.unique():
		if int(year) in cfg["year"]:
			fname = f"pp-{year}.csv"
			df[df.Year == year].to_csv(
				os.path.join(cfg["pp_processed dir"], fname), index=False
			)
			print(f"{fname} saved")
		else:
			print(
				f"Are you sure you want to process year {year}? Change configurations."
			)


# -------------------------------------------------------#
if cfg["process_raw_pp"]:
	for infile in cfg["raw price files"]:
		process_and_save_houseprice(infile, postcodes, postcode_region)

Processing pp-2024.csv
Number of transactions in pp-2024.csv: 641,336

pp-2024.csv saved
CPU times: user 4.62 s, sys: 369 ms, total: 4.99 s
Wall time: 4.99 s


In [57]:
cfg["pp_processed dir"]

'../input/HousePriceData/Processed'

#### Loading processed data

In [58]:
%%time


def load_processed_houseprice_data(year):
	fname = os.path.join(cfg["pp_processed dir"], f"pp-{str(year)}.csv")
	if os.path.isfile(fname):
		df = pd.read_csv(fname)
		print(f"Transaction count in {year}: {len(df):,}")
		return df
	else:
		return None


# ----------------------------------------#
with Pool(cpu_count() - 1) as p:
	results = p.map(load_processed_houseprice_data, cfg["Years"])

house_price_df = pd.concat(results, ignore_index=True)
del results

print("")
print(f"Number of rows in house_price_df: {len(house_price_df):,}")

Transaction count in 2008: 649,200
Transaction count in 2009: 624,926
Transaction count in 1995: 791,428
Transaction count in 1996: 958,613
Transaction count in 1998: 1,046,548
Transaction count in 2005: 1,060,732
Transaction count in 2000: 1,125,331
Transaction count in 1997: 1,088,301
Transaction count in 2001: 1,241,785
Transaction count in 1999: 1,190,704
Transaction count in 2004: 1,230,885
Transaction count in 2007: 1,271,602
Transaction count in 2003: 1,233,679
Transaction count in 2002: 1,348,336
Transaction count in 2006: 1,325,298
Transaction count in 2010: 662,919
Transaction count in 2011: 660,748
Transaction count in 2012: 668,335
Transaction count in 2013: 806,108
Transaction count in 2014: 971,344
Transaction count in 2015: 990,104
Transaction count in 2016: 999,686
Transaction count in 2017: 989,828
Transaction count in 2018: 963,597
Transaction count in 2019: 922,052
Transaction count in 2020: 700,217
Transaction count in 2021: 1,059,362
Transaction count in 2022: 651,

In [59]:
house_price_df.head(2)

,Price,Date,Post Code,Property Type,Old/New,Duration,Address,Post Code Coords,Year-Month,Year,Month,Sector,Region
0,17000,1995-01-01 00:00,HU5 5NY,T,N,F,147 WESTLANDS ROAD HULL HULL KINGSTON UPON HULL HUMBERSIDE,"[53.7536744770554, -0.414045315055496]",1995-01,1995,1,HU5 5,North England
1,61000,1995-01-01 00:00,CT9 5HW,D,N,F,15 WENTWORTH AVENUE WESTBROOK MARGATE THANET KENT,"[51.3821987268994, 1.3493702654861102]",1995-01,1995,1,CT9 5,South East


In [60]:
house_price_df.tail(2)

,Price,Date,Post Code,Property Type,Old/New,Duration,Address,Post Code Coords,Year-Month,Year,Month,Sector,Region
28429175,685500,2024-12-30 00:00,SW3 4HH,F,N,L,5 FLAT 4 CHEYNE PLACE LONDON KENSINGTON AND CHELSEA GREATER LONDON,"[51.4852308622622, -0.163027292802603]",2024-12,2024,12,SW3 4,Greater London
28429176,327500,2024-12-31 00:00,NG8 2QY,S,N,F,40 CRANSLEY AVENUE NOTTINGHAM CITY OF NOTTINGHAM CITY OF NOTTINGHAM,"[52.9454171649136, -1.22921795861809]",2024-12,2024,12,NG8 2,Midlands


#### Average price by month

In [61]:
temp = house_price_df[house_price_df["Property Type"] == "D"]

P = (
	house_price_df[["Year-Month", "Price"]]
	.groupby(by=["Year-Month"])
	.agg(["mean", "count"])
)
P.reset_index(inplace=True)
P.columns = ["Year-Month", "Price", "Volume"]

In [62]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
	px.line(
		P, x="Year-Month", y="Price", color_discrete_sequence=["cornflowerblue"]
	).data[0],
	secondary_y=True,
)
fig.add_trace(
	px.bar(P, x="Year-Month", y="Volume", color_discrete_sequence=["violet"]).data[0],
	secondary_y=False,
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(title_text="Avg. Price (£)", secondary_y=True)
fig.update_yaxes(
	title_text="Sales Volume (Bar Chart)", secondary_y=False, showgrid=False
)
fig.update_layout(
	title="England & Wales average house price and sales volume from 1995"
)

fig.show()

In [63]:
P2 = (
	house_price_df[["Year-Month", "Price", "Property Type"]]
	.groupby(by=["Year-Month", "Property Type"])
	.mean()
)
P2.reset_index(inplace=True)

fig = px.line(P2, x="Year-Month", y="Price", color="Property Type")
fig.update_layout(
	title="England & Wales average house price and sales volume from 1995 by Property Type"
)
fig.show()

#### Getting and saving average sector_price by year (for Chropleth)

In [64]:
def get_sector_df(house_price_df):
	P = (
		house_price_df[["Year", "Sector", "Price"]]
		.groupby(by=["Year", "Sector"])
		.agg(["mean", "count"])
	)
	P.reset_index(inplace=True)
	P.columns = ["Year", "Sector", "Price", "Volume"]

	P = P.loc[P["Sector"] != ""]

	# Get Region
	P["Region"] = P["Sector"].apply(lambda s: lookup_region(postcode_region, s))

	P["Price"] = P["Price"].apply(lambda s: int(np.round(s / 1000) * 1000))
	P["Display Price"] = P["Price"].apply(lambda x: f"{int(np.round(x / 1000)):,}K")
	P["text"] = (
		P["Sector"]
		+ "<br>"
		+ "Avg. Price: "
		+ P["Display Price"]
		+ "<br>"
		+ "Sales Volume: "
		+ P["Volume"].astype(str)
	)
	P.drop(columns=["Display Price"], inplace=True)

	return P

In [65]:
%%time
sector_df = get_sector_df(house_price_df)
sector_df.head()

CPU times: user 2.45 s, sys: 176 ms, total: 2.62 s
Wall time: 2.62 s


,Year,Sector,Price,Volume,Region,text
0,1995,AL1 1,90000,164,South East,AL1 1<br>Avg. Price: 90K<br>Sales Volume: 164
1,1995,AL1 2,81000,70,South East,AL1 2<br>Avg. Price: 81K<br>Sales Volume: 70
2,1995,AL1 3,89000,94,South East,AL1 3<br>Avg. Price: 89K<br>Sales Volume: 94
3,1995,AL1 4,128000,180,South East,AL1 4<br>Avg. Price: 128K<br>Sales Volume: 180
4,1995,AL1 5,73000,173,South East,AL1 5<br>Avg. Price: 73K<br>Sales Volume: 173


In [66]:
%%time

sector_by_year = dict()
for year in cfg["Years"]:
	sector_by_year[year] = sector_df[sector_df.Year == year].reset_index(drop=True)

	if cfg["to save"]:
		fname = os.path.join(cfg["appData dir"], f"sector_price_{year}.csv")
		sector_by_year[year].to_csv(fname, index=False)
		print(f"{fname} saved")

../appData/sector_price_1995.csv saved
../appData/sector_price_1996.csv saved
../appData/sector_price_1997.csv saved
../appData/sector_price_1998.csv saved
../appData/sector_price_1999.csv saved
../appData/sector_price_2000.csv saved
../appData/sector_price_2001.csv saved
../appData/sector_price_2002.csv saved
../appData/sector_price_2003.csv saved
../appData/sector_price_2004.csv saved
../appData/sector_price_2005.csv saved
../appData/sector_price_2006.csv saved
../appData/sector_price_2007.csv saved
../appData/sector_price_2008.csv saved
../appData/sector_price_2009.csv saved
../appData/sector_price_2010.csv saved
../appData/sector_price_2011.csv saved
../appData/sector_price_2012.csv saved
../appData/sector_price_2013.csv saved
../appData/sector_price_2014.csv saved
../appData/sector_price_2015.csv saved
../appData/sector_price_2016.csv saved
../appData/sector_price_2017.csv saved
../appData/sector_price_2018.csv saved
../appData/sector_price_2019.csv saved
../appData/sector_price_2

#### Getting and saving sector_percentage_delta by year (for Chropleth)

In [67]:
%%time

# Building sector_price[year] {sector: price} dict for quick lookup
sector_price = dict()
for year in cfg["Years"]:
	sector_price[year] = dict()
	for sector, region, price in sector_by_year[year][
		["Sector", "Region", "Price"]
	].values:
		sector_price[year][sector] = [region, price]

# -------------------------------#
sector_delta = dict()

sector_delta[1995] = dict()
for sector, [region, price] in sector_price[1995].items():
	sector_delta[1995][sector] = [0, region]

for y1, y2 in zip(cfg["Years"][1:], cfg["Years"][:-1]):
	sector_delta[y1] = dict()
	for sector, [region, price] in sector_price[y1].items():
		if sector in sector_price[y2]:
			last_year_price = sector_price[y2][sector][1]
			delta = int(np.round(100 * (price - last_year_price) / last_year_price))
			sector_delta[y1][sector] = [delta, region]

# ----------------------------------------------#
for year in cfg["Years"]:
	tmp = pd.DataFrame.from_dict(
		sector_delta[year], orient="index", columns=["Percentage Change", "Region"]
	)
	tmp.reset_index(inplace=True)
	tmp.rename(columns={"index": "Sector"}, inplace=True)
	tmp["text"] = (
		tmp["Sector"]
		+ "<br>"
		+ "Price Change: "
		+ tmp["Percentage Change"].apply(lambda s: str(s))
		+ "%"
	)

	if cfg["to save"]:
		fname = os.path.join(cfg["appData dir"], f"sector_percentage_delta_{year}.csv")
		tmp.to_csv(fname, index=False)
		print(f"{fname} saved")

../appData/sector_percentage_delta_1995.csv saved
../appData/sector_percentage_delta_1996.csv saved
../appData/sector_percentage_delta_1997.csv saved
../appData/sector_percentage_delta_1998.csv saved
../appData/sector_percentage_delta_1999.csv saved
../appData/sector_percentage_delta_2000.csv saved
../appData/sector_percentage_delta_2001.csv saved
../appData/sector_percentage_delta_2002.csv saved
../appData/sector_percentage_delta_2003.csv saved
../appData/sector_percentage_delta_2004.csv saved
../appData/sector_percentage_delta_2005.csv saved
../appData/sector_percentage_delta_2006.csv saved
../appData/sector_percentage_delta_2007.csv saved
../appData/sector_percentage_delta_2008.csv saved
../appData/sector_percentage_delta_2009.csv saved
../appData/sector_percentage_delta_2010.csv saved
../appData/sector_percentage_delta_2011.csv saved
../appData/sector_percentage_delta_2012.csv saved
../appData/sector_percentage_delta_2013.csv saved
../appData/sector_percentage_delta_2014.csv saved


#### Get Price and Volume by Year and Property Type df (For time-series)

In [68]:
%%time


def get_price_volume_df(house_price_df):
	cols = ["Year", "Sector", "Property Type"]
	P = house_price_df[cols + ["Price"]].groupby(by=cols).agg(["count", "mean"])
	P.reset_index(inplace=True)

	P.columns = ["Year", "Sector", "Property Type", "Count", "Average Price"]
	P = P.loc[P["Sector"] != ""]
	return P


# ------------------------------------#
price_volume_df = get_price_volume_df(house_price_df)
if cfg["to save"]:
	fname = os.path.join(cfg["appData dir"], "price_volume.csv")
	price_volume_df.to_csv(fname, index=False)
	print(f"{fname} saved")

../appData/price_volume.csv saved
CPU times: user 2.89 s, sys: 299 ms, total: 3.19 s
Wall time: 3.19 s


In [69]:
price_volume_df.head()

,Year,Sector,Property Type,Count,Average Price
0,1995,AL1 1,D,17,165320.588235
1,1995,AL1 1,F,55,64719.000000
2,1995,AL1 1,S,19,91328.947368
3,1995,AL1 1,T,73,91739.315068
4,1995,AL1 2,D,4,119375.000000


#### Regional Price data by year

In [70]:
regions = [r for r in house_price_df.Region.unique() if isinstance(r, str)]
print(f"Regions: {regions}")


def get_regional_price_data(sector_df, regions):
	def inner(region):
		if region == "South East":  # Include Greater London in South East graph
			mask = (sector_df.Region == region) | (sector_df.Region == "Greater London")
			df = sector_df[mask]
		else:
			df = sector_df[sector_df.Region == region]
		return df

	###########################################
	regional_price_data = dict()

	for r in regions:
		regional_price_data[r] = inner(r)

	return regional_price_data

Regions: ['North England', 'South East', 'Greater London', 'Midlands', 'South West', 'Wales', 'Scotland']


In [71]:
# Breaking price/volume data up by region:
regional_price_data = dict()
for year in cfg["Years"]:
	regional_price_data[year] = get_regional_price_data(sector_by_year[year], regions)

### Geo Data

#### Simplify the shapely file

In [72]:
%%time
# ref: https://shapely.readthedocs.io/en/stable/manual.html#object.simplify

if cfg["process geodata"]:
	# Let's enable speedups to make queries faster
	shapely.speedups.enable()

	infile = os.path.join(cfg["distribution dir"], "Sectors.shp")
	outfile = os.path.join(cfg["geodata dir"], "ukpostcode_geojson.json")

	shape_gdf = gpd.read_file(infile)

	polygons, discrepancies = [], []
	for i, x in enumerate(shape_gdf.geometry.values):
		tolerance = cfg["tolerance"]

		while True:
			y = x.simplify(tolerance, preserve_topology=True)
			discrepancy = 100 * np.abs(y.area - x.area) / x.area
			if discrepancy <= cfg["max discrepancy"]:
				break
			else:
				tolerance /= 2

		polygons.append(y)
		discrepancies.append(discrepancy)

		if i % 1000 == 0:
			print(f"{i:,} records processed")
			clear_output(wait=True)

	print(f"Average discrepancy: {np.mean(discrepancies):.2f}%")
	simplified_shape_gdf = deepcopy(shape_gdf)
	simplified_shape_gdf["geometry"] = polygons

	simplified_shape_gdf.to_file(outfile, driver="GeoJSON")
	simplified_shape_gdf.head()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.29 μs


In [73]:
# No longer needed - the simplified geojson version is much faster to load
# Convert Shape file to Geojson
# Code modified from https://github.com/akkana/scripts/blob/master/mapping/polidistmap.py

# infile  = os.path.join(cfg['distribution dir'], 'Sectors.shp')
# outfile = os.path.join(cfg['geodata dir'], 'ukpostcode_geojson.json')

# if not os.path.isfile(outfile):
#     options = gdal.VectorTranslateOptions(format="GeoJSON", dstSRS="EPSG:4326")
#     gdal.VectorTranslate(outfile, infile, options=options)
#     print("Translated GEOJSON file", outfile)

In [74]:
def load_geo_data(infile):
	with open(infile, "r") as read_file:
		geo_data = json.load(read_file)
	return geo_data


# ---------------------------------------------#
if cfg["process geodata"]:
	infile = os.path.join(cfg["geodata dir"], "ukpostcode_geojson.json")
	geo_data = load_geo_data(infile)

In [75]:
def get_regional_geo_data(geo_data, postcode_region, regions=cfg["Regions"]):
	pattern = re.compile(r"\d")

	def inner(region):
		Y = dict()
		Y["features"] = []
		for k in geo_data.keys():
			if k != "features":
				Y[k] = geo_data[k]
			else:
				for i, d in enumerate(geo_data["features"]):
					for k, v in d.items():
						if k == "properties":
							sector = v["name"]
							m = pattern.search(sector)
							district = sector[: m.start()]

							if region == "South East":
								if postcode_region[district] in [
									region,
									"Greater London",
								]:
									Y["features"].append(geo_data["features"][i])
							else:
								if postcode_region[district] == region:
									Y["features"].append(geo_data["features"][i])
		return Y

	###########################################
	regional_geo_data = dict()
	for r in regions:
		regional_geo_data[r] = inner(r)

	return regional_geo_data

In [76]:
%%time

if cfg["process geodata"]:
	# Breaking geo_data up by region:
	regional_geo_data = get_regional_geo_data(geo_data, postcode_region)

	for region, value in regional_geo_data.items():
		fname = os.path.join(cfg["assets dir"], f"geodata_{region}.json")
		with open(fname, "w") as f:
			json.dump(value, f)

CPU times: user 1 μs, sys: 1 μs, total: 2 μs
Wall time: 3.58 μs


#### School Data
https://www.gov.uk/school-performance-tables

In [77]:
regnumber = re.compile(r"\d+")


def num_2_str(x):
	if x is not None and regnumber.match(x):
		return float(x)
	else:
		return 0.0

In [78]:
# Import GCSE and ALevel data, filter and basic cleaning

fields = ["URN", "SCHNAME", "PCODE", "EGENDER", "AGERANGE", "ATT8SCR"]
gcse_df = pd.read_csv(
	os.path.join(cfg["school dir"], "england_ks4final.csv"),
	usecols=fields,
	dtype={"URN": str},
)
gcse_df.dropna(inplace=True)
gcse_df["ATT8SCR"] = gcse_df["ATT8SCR"].apply(lambda x: num_2_str(x))
gcse_df = gcse_df[gcse_df["ATT8SCR"] > 0]
gcse_df.sort_values(by=["ATT8SCR"], ascending=False, ignore_index=True, inplace=True)
gcse_df["GCSE rank"] = gcse_df.index + 1
gcse_df.rename(columns={"EGENDER": "GENDER"}, inplace=True)

print(f"gcse_df: {gcse_df.shape}")

fields = ["URN", "SCHNAME", "PCODE", "GEND1618", "AGERANGE", "TALLPPE_ALEV_1618"]
alevel_df = pd.read_csv(
	os.path.join(cfg["school dir"], "england_ks5final.csv"),
	usecols=fields,
	dtype={"URN": str},
)
alevel_df.dropna(inplace=True)
alevel_df["TALLPPE_ALEV_1618"] = alevel_df["TALLPPE_ALEV_1618"].apply(
	lambda x: num_2_str(x)
)
alevel_df = alevel_df[alevel_df["TALLPPE_ALEV_1618"] > 0]
alevel_df.sort_values(
	by=["TALLPPE_ALEV_1618"], ascending=False, ignore_index=True, inplace=True
)
alevel_df["A-Level rank"] = alevel_df.index + 1
alevel_df.rename(columns={"GEND1618": "GENDER"}, inplace=True)

print(f"alevel_df: {alevel_df.shape}")

gcse_df: (4522, 7)
alevel_df: (2624, 7)


In [79]:
def merge_df(df_x, df_y, key):
	col_x = df_x.columns
	col_y = df_y.columns
	cols = set(col_x).intersection(set(col_y))
	cols.discard(key)

	df_z = df_x.merge(df_y, how="outer", on=key)

	for col in cols:
		tmp = []
		C = [f"{col}_x", f"{col}_y"]
		for a, b in df_z[C].values:
			if isinstance(a, str):
				tmp.append(a)
			else:
				tmp.append(b)
		df_z[col] = tmp
		df_z.drop(columns=[*C], inplace=True)

	return df_z

In [80]:
# Merge GCSE and ALevel data into school_df
school_df = merge_df(gcse_df, alevel_df, "URN")
school_df["GENDER"] = school_df["GENDER"].apply(lambda s: s.capitalize())

In [81]:
# Get best of GCSE and Alevel Rank:
tmp = []
for gcse_rank, alevel_rank in school_df[["GCSE rank", "A-Level rank"]].values:
	tmp.append(int(np.nanmin([gcse_rank, alevel_rank])))

school_df["Best Rank"] = tmp

In [82]:
# Merge school status into school_df
school_info_df = pd.read_csv(
	os.path.join(cfg["school dir"], "england_school_information.csv"),
	dtype={"URN": str},
)
school_df = school_df.merge(school_info_df[["URN", "MINORGROUP"]], how="left", on="URN")
school_df.rename(columns={"MINORGROUP": "Status"}, inplace=True)

In [83]:
# Merge lat, long into school_df
fields = ["URN", "Easting", "Northing"]
school_loc_df = pd.read_csv(
	os.path.join(cfg["geodata dir"], "EduBase Extract - 2016-0005414.csv"),
	usecols=fields,
	encoding="ISO-8859-1",
	dtype={"URN": str},
)

school_df = school_df.merge(
	school_loc_df[["URN", "Easting", "Northing"]], how="left", on="URN"
)
school_df = school_df[pd.notna(school_df.Easting) | pd.notna(school_df.Northing)]

long, lat = convert_lonlat(school_df.Easting.values, school_df.Northing.values)
school_df["Latitude"] = lat
school_df["Longitude"] = long
school_df.drop(columns=["Easting", "Northing"], inplace=True)

In [84]:
# Get Hover label text
tmp = []
cols = [
	"SCHNAME",
	"AGERANGE",
	"GENDER",
	"Status",
	"ATT8SCR",
	"GCSE rank",
	"TALLPPE_ALEV_1618",
	"A-Level rank",
]

for row in school_df[cols].values:
	text = row[0] + "<br>" + row[1] + " " + row[2] + " " + row[3] + " "
	if ~np.isnan(row[4]):
		text += "<br>" + "GCSE: A8S " + str(row[4]) + ", #" + f"{int(row[5]):,}"
	if ~np.isnan(row[6]):
		text += "<br>" + "A-level: APS " + str(row[6]) + ", #" + f"{int(row[7]):,}"

	tmp.append(text)

school_df["Info"] = tmp

cols.remove("ATT8SCR")
cols.remove("TALLPPE_ALEV_1618")
school_df.drop(columns=cols, inplace=True)

In [85]:
school_df.head(2)

,URN,ATT8SCR,TALLPPE_ALEV_1618,PCODE,Best Rank,Latitude,Longitude,Info
0,100001,57.9,51.02,EC2Y 8BB,13,51.519149,-0.094540,"City of London School for Girls<br>7-18 Girls Independent school <br>GCSE: A8S 57.9, #384<br>A-level: APS 51.02, #13"
1,100003,20.0,49.41,EC4V 3AL,27,51.511118,-0.099487,"City of London School<br>10-18 Boys Independent school <br>GCSE: A8S 20.0, #3,787<br>A-level: APS 49.41, #27"


In [86]:
for n in [500]:
	gcse_df = school_df.sort_values(by=["ATT8SCR"], ascending=False, ignore_index=True)[
		:n
	]
	alevel_df = school_df.sort_values(
		by=["TALLPPE_ALEV_1618"], ascending=False, ignore_index=True
	)[:n]

	school_topN = pd.concat([gcse_df, alevel_df])
	school_topN.drop_duplicates(subset=["URN"], inplace=True)
	school_topN["Region"] = school_topN["PCODE"].apply(
		lambda x: lookup_region(postcode_region, x)
	)

	print(school_topN.shape)

	fname = os.path.join(cfg["appData dir"], f"schools_top_{n}.csv")
	school_topN.to_csv(fname, index=False)

(784, 9)


In [87]:
Regions = [
	"South East",
	"North England",
	"Midlands",
	"South West",
	"Greater London",
	"Wales",
]

for region in Regions:
	if region != "South East":
		mask = school_topN.Region == region
	else:
		mask = (school_topN.Region == "South East") | (
			school_topN.Region == "Greater London"
		)

	fname = os.path.join(cfg["appData dir"], f"schools_{region}.csv")
	school_topN[mask].to_csv(fname, index=False)

In [88]:
print(f"Data Preparation completed in {(time.time() - t0) / 60:.1f} minutes")
print(datetime.datetime.now())

Data Preparation completed in 0.7 minutes
2025-03-16 21:32:44.745709
